In [1]:
import numpy as np
import pandas as pd
import math
from datetime import datetime

import sys
import bokeh

from IPython import __version__ as ipython_version
from pandas import __version__ as pandas_version
from bokeh import __version__ as bokeh_version
print("IPython - %s" % ipython_version)
print("Pandas - %s" % pandas_version)
print("Bokeh - %s" % bokeh_version)

IPython - 7.6.1
Pandas - 0.24.2
Bokeh - 2.0.1


In [2]:
# Standard imports 
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [3]:
# Bokeh tools
from bokeh.models import ColumnDataSource, ColorBar, HoverTool, Legend
from bokeh.palettes import brewer
from bokeh.layouts import row, column, gridplot
from bokeh.models import CustomJS, Slider, Select, Plot, Button, LinearAxis, Range1d, DatetimeTickFormatter
from bokeh.models.glyphs import Line, MultiLine

# Read datasets

In [4]:
df_confirmed_global_raw = pd.read_csv('datasets/time_series_covid19_confirmed_global.csv')
df_confirmed_global_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,906,933,996,1026,1092,1176,1279,1351,1463,1531
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,539,548,562,584,609,634,663,678,712,726
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2418,2534,2629,2718,2811,2910,3007,3127,3256,3382
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,696,704,713,717,717,723,723,731,738,738
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,19,24,24,24,24,25,25,25,25,26


In [5]:
df_recovered_global_raw = pd.read_csv('datasets/time_series_covid19_recovered_global.csv')
df_recovered_global_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,99,112,131,135,150,166,179,188,188,207
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,283,302,314,327,345,356,385,394,403,410
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,846,894,1047,1099,1152,1204,1355,1408,1479,1508
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,191,205,235,248,282,309,333,344,344,344
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,5,6,6,6,6,6,6,6,6,6


In [6]:
df_deaths_global_raw = pd.read_csv('datasets/time_series_covid19_deaths_global.csv')
df_deaths_global_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,30,30,33,36,36,40,42,43,47,50
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,26,26,26,26,26,27,27,27,27,28
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,364,367,375,384,392,402,407,415,419,425
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,35,35,36,37,37,37,37,40,40,40
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


# Data Preparation

In [7]:
def to_datetime(date):
    return datetime.strptime(date, '%m/%d/%y')

In [8]:
df_confirmed_global = df_confirmed_global_raw.groupby('Country/Region').sum().drop(columns=['Lat', 'Long'])
df_confirmed_global = df_confirmed_global.T.reset_index().rename(columns={'index':'date'})
df_confirmed_global['date'] = df_confirmed_global['date'].apply(lambda x: to_datetime(x))
df_confirmed_global.tail()

Country/Region,date,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
91,2020-04-22,1176,634,2910,723,25,24,3144,1473,6652,...,134638,543,1716,288,268,474,6,1,74,28
92,2020-04-23,1279,663,3007,723,25,24,3435,1523,6662,...,139246,557,1758,311,268,480,6,1,76,28
93,2020-04-24,1351,678,3127,731,25,24,3607,1596,6677,...,144640,563,1804,318,270,484,6,1,84,29
94,2020-04-25,1463,712,3256,738,25,24,3780,1677,6694,...,149569,596,1862,323,270,342,6,1,84,31
95,2020-04-26,1531,726,3382,738,26,24,3892,1746,6714,...,154037,606,1869,325,270,342,6,1,88,31


In [9]:
df_recovered_global = df_recovered_global_raw.groupby('Country/Region').sum().drop(columns=['Lat', 'Long'])
df_recovered_global = df_recovered_global.T.reset_index().rename(columns={'index':'date'})
df_recovered_global['date'] = df_recovered_global['date'].apply(lambda x: to_datetime(x))
df_recovered_global.tail()

Country/Region,date,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
91,2020-04-22,166,356,1204,309,6,10,872,633,4932,...,683,324,450,122,223,71,0,0,35,2
92,2020-04-23,179,385,1355,333,6,10,919,659,5047,...,712,354,561,126,224,92,5,0,37,2
93,2020-04-24,188,394,1408,344,6,11,976,728,5136,...,724,369,621,132,220,92,5,1,37,2
94,2020-04-25,188,403,1479,344,6,11,1030,803,5376,...,774,370,707,132,225,92,5,1,37,2
95,2020-04-26,207,410,1508,344,6,11,1107,833,5541,...,778,375,789,137,225,83,5,1,42,2


In [10]:
df_deaths_global = df_deaths_global_raw.groupby('Country/Region').sum().drop(columns=['Lat', 'Long'])
df_deaths_global = df_deaths_global.T.reset_index().rename(columns={'index':'date'})
df_deaths_global['date'] = df_deaths_global['date'].apply(lambda x: to_datetime(x))
df_deaths_global.tail()

Country/Region,date,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
91,2020-04-22,40,27,402,37,2,3,152,24,67,...,18151,12,7,10,0,4,0,0,3,4
92,2020-04-23,42,27,407,37,2,3,165,24,75,...,18791,12,7,10,0,4,0,0,3,4
93,2020-04-24,43,27,415,40,2,3,176,27,79,...,19567,12,8,10,0,4,0,0,3,4
94,2020-04-25,47,27,419,40,2,3,185,28,80,...,20381,14,8,10,0,2,0,0,3,4
95,2020-04-26,50,28,425,40,2,3,192,28,83,...,20794,15,8,10,0,2,0,0,3,4


In [11]:
df_global = df_confirmed_global.copy()
df_global = df_global[['date', 'Afghanistan']].drop('Afghanistan', axis=1)
df_global['confirmed'] = df_confirmed_global.iloc[:,1:].sum(axis=1)
df_global['recovered'] = df_recovered_global.iloc[:,1:].sum(axis=1)
df_global['deaths'] = df_deaths_global.iloc[:,1:].sum(axis=1)
df_global.head()

Country/Region,date,confirmed,recovered,deaths
0,2020-01-22,555,28,17
1,2020-01-23,654,30,18
2,2020-01-24,941,36,26
3,2020-01-25,1434,39,42
4,2020-01-26,2118,52,56


# Plot Global

In [12]:
def plot_global():
    source = ColumnDataSource(df_global)
    
    query = ['date', 'confirmed']
    data_filter = df_global[query].rename(columns={'confirmed' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Total Case",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Total Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(y[y.length - 1]+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    options = ['confirmed', 'recovered', 'deaths']
    
    # define select
    select = Select(title="Options:", value='Indonesia', options=options)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    show(layout)
    
plot_global()

# Plot Confirmed Global by Country

In [13]:
def plot_confirmed_global():
    source = ColumnDataSource(df_confirmed_global)
    
    query = ['date', 'Indonesia']
    data_filter = df_confirmed_global[query].rename(columns={'Indonesia' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Confirmed Case",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Total Confirmed Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(y[y.length - 1]+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    countries = df_confirmed_global_raw['Country/Region'].unique().tolist()
    
    # define select
    select = Select(title="Confirmed Case:", value='Indonesia', options=countries)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    show(layout)
    
plot_confirmed_global()

# Plot Recovered Global by Country

In [14]:
def plot_recovered_global():
    source = ColumnDataSource(df_recovered_global)
    
    query = ['date', 'Indonesia']
    data_filter = df_recovered_global[query].rename(columns={'Indonesia' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Recovered Global",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Total Confirmed Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(y[y.length - 1]+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    countries = df_confirmed_global_raw['Country/Region'].unique().tolist()
    
    # define select
    select = Select(title="Recovered Global:", value='Indonesia', options=countries)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    show(layout)
    
plot_recovered_global()

# Plot Deaths Global by Country

In [15]:
def plot_deaths_global():
    source = ColumnDataSource(df_deaths_global)
    
    query = ['date', 'Indonesia']
    data_filter = df_deaths_global[query].rename(columns={'Indonesia' : 'cases'})
    scr_data = ColumnDataSource(data_filter)
    
    hover_tool = HoverTool(tooltips=[('Cases', '@cases'), ('Date', '@date')],
                           formatters={'date': 'datetime'}
                          )
    
    p = figure(x_axis_type="datetime",
               title="Deaths Global",
               plot_height=500, plot_width=700,
               tools=[hover_tool],
               y_range=Range1d(start=0, end=data_filter.cases.max()+50)
              )
    
    p.line(x='date',y='cases', source=scr_data, line_width=3, line_alpha=.8)
    
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Total Confirmed Case'
    
    p.background_fill_color = "whitesmoke"
    p.background_fill_alpha = 0.5
    
    p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m", months="%m/%d %H:%M",)
    
    #this javascript snippet is the callback when either select is changed
    code = """
    var c = cb_obj.value;
    ax.axis_label = c;
    var y = s1.data[c];
    s2.data['cases'] = y;
    y_range.start = 0;
    y_range.end = parseInt(y[y.length - 1]+50);
    s2.change.emit();
    """
    
    # define callback
    callback = CustomJS(args=dict(s1=source, s2=scr_data, y_range=p.y_range,ax=p.yaxis), code=code)

    # country list
    countries = df_confirmed_global_raw['Country/Region'].unique().tolist()
    
    # define select
    select = Select(title="Deaths Global:", value='Indonesia', options=countries)
    select.js_on_change('value', callback)

    btn = Button(label='Update')

    layout = column(row(select), row(p))
    show(layout)
    
plot_deaths_global()